In [30]:
import numpy as np
import pyregion
import astropy.io.fits as pyfits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS

from sys import argv
import warnings
import time
import os

In [31]:
#---------------------------------------
# Input some parameters from parameter.ipynb
#---------------------------------------
%run parameter.ipynb


#---------------------------------------
# Input parameters used in this program:

###-------------------------------------
### Path name & outname & imageName
# path_image, path_BT, path_figure, path_samping, path_region
# outname_b4,  outname_b7


In [34]:
#----------------------------------------
# define the functions
#----------------------------------------

def PA_region(input_nqtxt_name, output_reg_name, seg_color, seg_width, 
              scale_10percent, scale_length, IS_Per, Is_E_field):

    nyquist_list = np.loadtxt(input_nqtxt_name, dtype = str)
    num, b = np.shape(nyquist_list)

    # Open the output regoin file and write the header
    reg_file = open(output_reg_name, "w")
    reg_file.write(
        '# Region file format: DS9 version 4.1\n'
        'global color={0} dashlist=8 3 width={1} font="helvetica 10 normal" '
        'select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 '
        'source=1\nfk5\n'.format(seg_color, seg_width))

    for i in xrange(num):
        # 0:RA, 2:Dec, 4:I, 6:Q, 8:U, 10:PI, 12:PA, 14:Per 
        ra = np.float(nyquist_list[i][0])                  # [deg]
        dec = np.float(nyquist_list[i][2])                 # [deg]
        Per = np.float(nyquist_list[i][14])                # [%]
        
        # Set the polarization angle 
        PA_E_deg = np.float(nyquist_list[i][12])           # PA [deg]
        PA_deg = PA_E_deg if Is_E_field == True else PA_E_deg + 90.       
        PA_rad = np.deg2rad(PA_deg)                        # PA [rad]
        
        # Set the scale length [arcsec]
        scale = scale_10percent if IS_Per == True else scale_length
           
        # Calculate the region as the polarization angle line segment
        # 0.5 * (polI/I) * sin[or cos](PA) * (10*scale_10percent[arcs])/3600 in deg
        # dRA is corrected by cos(dec)
        dx_half = 0.5 / 360 * scale * Per * np.sin(PA_rad) / np.cos(np.deg2rad(dec)) 
        dy_half = 0.5 / 360 * scale * Per * np.cos(PA_rad)
        
        # write the region
        reg_file.write("line({0},{1},{2},{3}) # line=0 0 # width=2\n".format(
                        ra + dx_half, dec + dy_half, ra - dx_half, dec - dy_half))
            
    reg_file.close()
    
#---------------------------------------    
def make_region(outname, scale_10percent, scale_length, IS_Per, \
                seg_width, seg_width_base, seg_color, seg_color_base, reg_parm):

    # region filename
    nyquist_name = '%snyquist_%s_%s.txt'%(path_samping, outname, reg_parm)
    output_E_reg_name = '%s%s_%s_E_PA.reg'%(path_region, outname, reg_parm)
    output_E_reg_base_name = '%s%s_%s_E_PA_base.reg'%(path_region, outname, reg_parm)
    output_B_reg_name = '%s%s_%s_B_PA.reg'%(path_region, outname, reg_parm)
    output_B_reg_base_name = '%s%s_%s_B_PA_base.reg'%(path_region, outname, reg_parm)

    # E-field line segments 
    PA_region(nyquist_name, output_E_reg_name, seg_color, seg_width, \
              scale_10percent, scale_length, IS_Per, Is_E_field=True)
    PA_region(nyquist_name, output_E_reg_base_name, seg_color, seg_width, \
              scale_10percent, scale_length, IS_Per, Is_E_field=True)
    # B-field line segments
    PA_region(nyquist_name, output_B_reg_name, seg_color, seg_width, \
              scale_10percent, scale_length, IS_Per, Is_E_field=False)
    PA_region(nyquist_name, output_B_reg_base_name, seg_color, seg_width, \
              scale_10percent, scale_length, IS_Per, Is_E_field=False)

In [36]:
# Star measure time
start_time = time.time()

#---------------------------------------    
# Create the ds9 region file for plotting the Polarization angle
#---------------------------------------

#---------------------------------------    
# set some parameters

outnames = [
            outname_b4, outname_b7
            ]

IS_Per = False

for outname in outnames:
    
    # Get the seg parameter
    scale_10percent, scale_line, seg_color, seg_width, \
    seg_color_base, seg_width_base = slt_seg(outname) 

    #---------------------------------------
    # make the region with different criteria
    # (1) all the region (4A1 & 4A2)
    make_region(outname, scale_10percent, scale_length, IS_Per, \
                seg_width, seg_width_base, seg_color, seg_color_base, \
                reg_parm = 'all')
    # (2) masked region (4A1 & 4A2)
    make_region(outname, scale_10percent, scale_length, IS_Per, \
                seg_width, seg_width_base, seg_color, seg_color_base, \
                reg_parm = 'mask')
    # (3) 4A1 only
    make_region(outname, scale_10percent, scale_length, IS_Per, \
                seg_width, seg_width_base, seg_color, seg_color_base, \
                reg_parm = 'mask_4A1')
    # (4) 4A2 only
    make_region(outname, scale_10percent, scale_length, IS_Per, \
                seg_width, seg_width_base, seg_color, seg_color_base, \
                reg_parm = 'mask_4A2')
    # (5) innermost 100au region of 4A1
    make_region(outname, scale_10percent, scale_length, IS_Per, \
                seg_width, seg_width_base, seg_color, seg_color_base, \
                reg_parm = 'mask_4A1_inner100au', )
    # (6) innermost 100au region of 4A2
    make_region(outname, scale_10percent, scale_length, IS_Per, \
                seg_width, seg_width_base, seg_color, seg_color_base, \
                reg_parm = 'mask_4A2_inner100au', )
    # (7) nearby region without innermost 100au of 4A1
    make_region(outname, scale_10percent, scale_length, IS_Per, \
                seg_width, seg_width_base, seg_color, seg_color_base, \
                reg_parm = 'mask_4A1_wo_inner100au', )
    # (8) nearby region without innermost 100au of 4A2
    make_region(outname, scale_10percent, scale_length, IS_Per, \
                seg_width, seg_width_base, seg_color, seg_color_base, \
                reg_parm = 'mask_4A2_wo_inner100au', )


#---------------------------------------    
# End measure time
elapsed_time = time.time() - start_time
print ('#----------------------------------------')
print ("Done! Time spending = ", elapsed_time, "seconds.")

#----------------------------------------
('Done! Time spending = ', 2.211215019226074, 'seconds.')
